In [5]:
%matplotlib inline
import pandas as pd, matplotlib.pyplot as plt, mpld3
import seaborn as sn, numpy as np, glob, os
sn.set_context('notebook')

# Calculating median Ca and Mg for ICPW TOC trends

This notebook documents the calculation of **median (Ca + Mg)** values from the ICPW data in the RESAII database. The workflow is as follows:

## 1. Extract data

From RESAII, extract **sea-salt corrected ECa\* and EMg\*** for all the sites associated with projects named *ICPW_TOCTRENDS_2015_XX*, **except** the projects associated with river sites or those marked as "excluded". The time period of interest runs from **01/01/1990** to **31/12/2012**.

Unfortunately, RESAII throws a memory error if I try to download this data all at once, so instead download it separately for each country. The raw data files are here:

`\James_Work\Staff\Heleen_d_W\ICP_Waters\Ca_Mg_for_DonM\Data`

First, read the files and combine all the data.

**Note:** The code below prints out the list of RESAII project names that I've included in this analysis. **Ask Heleen/Don to check these** to make sure they are correct.

In [6]:
data_fold = r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Heleen_d_W\ICP_Waters\Ca_Mg_for_DonM\Data'

# Get list of files to process
search_path = os.path.join(data_fold, '*.xlsx')
file_list = glob.glob(search_path)

# Loop over files
df_list = []
print 'Projects from RESAII included:'
for file_path in file_list:
    # Print the name of the project in RESAII
    prj_name = 'ICPW_TOCTRENDS_2015_' + os.path.split(file_path)[1][:-5].upper()
    print '    ' + prj_name    
    
    # Read data
    df = pd.read_excel(file_path, sheetname='DATA')
    
    # Drop the blank first row
    df.drop(df.index[0], inplace=True)
    
    # Extract columns of interest
    cols = ['Station ID', 'Station Code', 'Station name', 'Date', 'ECa*', 'EMg*']
    df = df[cols]
    
    # Convert columns to correct data type
    df['Station ID'] = df['Station ID'].astype(int)
    df['ECa*'] = df['ECa*'].astype(float)
    df['EMg*'] = df['EMg*'].astype(float)

    # Add to list
    df_list.append(df)

# Concat data
df = pd.concat(df_list, axis=0)
df.reset_index(drop=True, inplace=True)

# Calculate (Ca + Mg)
df['ECa*+EMg*'] = df['ECa*'] + df['EMg*']

# Add column for year
df['Year'] = pd.DatetimeIndex(df['Date']).year

print '\n'
print 'Total number of records:', len(df)
df.head(10)

Projects from RESAII included:
    ICPW_TOCTRENDS_2015_CA_ATL
    ICPW_TOCTRENDS_2015_CA_DO
    ICPW_TOCTRENDS_2015_CA_ICPW
    ICPW_TOCTRENDS_2015_CA_NF
    ICPW_TOCTRENDS_2015_CA_QU
    ICPW_TOCTRENDS_2015_CZ
    ICPW_TOCTRENDS_2015_CZ2
    ICPW_TOCTRENDS_2015_FL
    ICPW_TOCTRENDS_2015_NO
    ICPW_TOCTRENDS_2015_SE
    ICPW_TOCTRENDS_2015_UK
    ICPW_TOCTRENDS_2015_US_LTM
    ICPW_TOCTRENDS_2015_US_TIME


Total number of records: 58257


,Station ID,Station Code,Station name,Date,ECa*,EMg*,ECa*+EMg*,Year
0,37281,X15:NS01DA0002,TEDFORD LAKE,1990-06-05,67.004,5.215,72.219,1990
1,37281,X15:NS01DA0002,TEDFORD LAKE,1990-10-30,56.919,13.711,70.630,1990
2,37281,X15:NS01DA0002,TEDFORD LAKE,1991-05-31,58.693,11.593,70.286,1991
3,37281,X15:NS01DA0002,TEDFORD LAKE,1992-06-03,66.169,4.083,70.252,1992
4,37281,X15:NS01DA0002,TEDFORD LAKE,1992-10-22,55.876,4.070,59.946,1992
5,37281,X15:NS01DA0002,TEDFORD LAKE,1993-07-05,56.084,0.000,56.084,1993
6,37281,X15:NS01DA0002,TEDFORD LAKE,1993-10-19,55.458,3.503,58.961,1993
7,37281,X15:NS01DA0002,TEDFORD LAKE,1994-11-03,52.784,32.331,85.115,1994
8,37281,X15:NS01DA0002,TEDFORD LAKE,1995-05-25,54.819,4.932,59.751,1995
9,37281,X15:NS01DA0002,TEDFORD LAKE,1995-10-17,44.962,9.111,54.073,1995


Next, group by station ID. As a check, I'll also try grouping by [Station ID, Station Code, Station name]. If all is well, these two options should give the same result.

In [7]:
# Get two possible grouping
grpd1 = df.groupby(['Station ID',])
grpd2 = df.groupby(['Station ID', 'Station Code', 'Station name'])

# Check the groups are the same size
assert len(grpd1) == len(grpd2), 'Groups are not the same size.'

# Extract just site, year and (Ca+Mg) columns
df2 = df[['Station ID', 'Station Code', 'Station name', 'Year', 'ECa*+EMg*']]

# Group
grpd = df2.groupby(['Station ID', 'Station Code', 'Station name'])

# Calculate stats.
agg = grpd.aggregate(['min', 'median', 'max', 'count'])

print 'Total number of sites:', len(agg)
agg.head(10)

Total number of sites: 519


Year                     ECa*+EMg*  \
                                           min  median   max count       min   
Station ID Station Code Station name                                           
100        623-603      Breidlivatnet     1990  2000.5  2012    22    19.411   
101        301-605      Langvann          1990  2001.0  2012    37    52.532   
102        221-605      Store Lyseren     1990  2002.0  2012    30    54.793   
103        914-501      Sandvatn          1990  2001.0  2012    25    30.925   
104        137-501      Ravnsjøen         1990  2002.0  2012    30    52.512   
105        221-607      Holvatn           1990  2000.5  2012    24    71.473   
107        819-501      Nedre Furuvatn    1990  2002.0  2012    29    29.615   
108        LAE01        Langtjern, utløp  1990  2002.0  2012  1207    20.758   
109        1021-14      Homestadvatnet    1990  2000.5  2012    24    16.018   
110        1845-601     Tennvatn          1990  2000.0  2012    25    14.017   

                                                                  
                                           median      max count  
Station ID Station Code Station name                              
100        623-603      Breidlivatnet     24.8115   34.335    22  
101        301-605      Langvann          63.0630   72.560    36  
102        221-605      Store Lyseren     69.7820  104.229    30  
103        914-501      Sandvatn          48.9280   88.876    25  
104        137-501      Ravnsjøen         68.8945  111.462    30  
105        221-607      Holvatn           86.6480  120.697    24  
107        819-501      Nedre Furuvatn    51.9300   71.574    29  
108        LAE01        Langtjern, utløp  58.6520  108.243  1168  
109        1021-14      Homestadvatnet    21.5630   28.014    24  
110        1845-601     Tennvatn          23.3050   30.505    25

This implies that there are **519** unique sites here. Note that the total number of sites in the RESAII database associated with these projects is **697**. I assume the discrepancy is because not all of these sites are associated with ECa\* and EMg\* values, but it might be worth double-checking to make sure this is not a database error. 

Note also that the number of samples for each site varies significantly, which may have implications for subsequent analyses.

For now, as Don needs the data quickly, I'll write the output to CSV for a quick check in Excel.

In [8]:
# Write output
out_xls = r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Heleen_d_W\ICP_Waters\Ca_Mg_for_DonM\ca_mg_medians.csv'

agg.to_csv(out_xls, encoding='utf-8')